In [1]:
import os

from joblib import Parallel, delayed
from tqdm import tqdm_notebook as tqdm
import pickle
import pandas as pd

import jams
import numpy as np

import muda

In [2]:
def root(x):
    return os.path.splitext(os.path.basename(x))[0]

AUDIO = jams.util.find_with_extension('/home/bmcfee/data/eric_chords/audio/', 'mp3')
ANNOS = jams.util.find_with_extension('/home/bmcfee/data/eric_chords/references_v2/', 'jams')

# Make sure there are the same number of files
assert len(AUDIO) == len(ANNOS)

# And that they're in agreement
assert all([root(_1) == root(_2) for (_1, _2) in zip(AUDIO, ANNOS)])

In [3]:
OUTDIR = '/home/bmcfee/working/chords/augmentation/'

In [4]:
!mkdir -p {OUTDIR}

In [5]:
def augment(afile, jfile, deformer, outpath):
    
    jam = muda.load_jam_audio(jfile, afile)
    
    base = root(afile)
    outfile = os.path.join(outpath, base)
    
    for i, jam_out in tqdm(enumerate(deformer.transform(jam))):    
        muda.save('{}.{}.ogg'.format(outfile, i),
                  '{}.{}.jams'.format(outfile, i),
                 jam_out, strict=False)

In [6]:
# Create the augmentation engine
pitcher = muda.deformers.PitchShift(n_semitones=[-1, -0.5, 0.5, 1])

In [7]:
with open('/home/bmcfee/working/chords/muda.pkl', 'wb') as fd:
    pickle.dump(pitcher, fd)

In [ ]:
augment(AUDIO[0], ANNOS[0], pitcher, OUTDIR)

In [ ]:
Parallel(n_jobs=20, verbose=10)(delayed(augment)(aud, jam, pitcher, OUTDIR) for (aud, jam) in zip(AUDIO, ANNOS));